## Лаба 2. Content-based рекомендательная система образовательных курсов – Spark Dataframes

#### Дедлайн

Воскресенье,  23 октября до 23:59:00(МСК).

#### Дедлайн Github

Среда, 26 октября до 23:59:00(МСК).

#### Задача

По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам. Запрещено использовать библиотеки pandas, sklearn и аналогичные.

#### Описание данных

Имеются следующие данные на вход:

* набор данных о всех курсах. Датасет можно взять с HDFS по адресу: `/labs/slaba02/DO_record_per_line.json`
* id курсов, для которых надо дать рекомендации (указаны в [Личном кабинете](https://lk-spark.newprolab.com/lab/slaba02)).

Данные выглядят следующим образом:

```json
{"lang": "en",
"name": "Accounting Cycle: The Foundation of Business Measurement and Reporting",
"cat": "3/business_management|6/economics_finance",
"provider": "Canvas Network",
"id": 4,
"desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close accounts, and how to prepare key financial reports. All material for this class is written and delivered by the professor, and can be previewed here. Students must have access to a spreadsheet program to participate."}
```

#### Результат

Для каждого id курса из личного кабинета необходимо дать топ-10 наиболее похожих на него курсов. Рекомендованные курсы должны быть того же языка, что и курс, для которого строится рекомендация.

Выходной формат — json — должен иметь следующую структуру:

```
{
  "123": [
    5372,
    16663,
    23114,
    13079,
    13084,
    ...
  ],
  "456": [
    ...
  ],
  "789": [
    ...
  ],
  "123456": [
    ...
  ],
  "456789": [
    ...
  ],
  "987654": [
    ...
  ]
}
```

Ключи json — это id курсов, для которых строится рекомендация. Для каждого такого ключа в качестве значения задается массив рекомендованных курсов, состоящий из их id, отсортированных по убыванию метрики. При равенстве значений метрики курсов сортируются лексикографически по названию.

Также возможна очень редкая ситуация (в основном с русскоязычными курсами), когда в рекомендацию попадут два дубликата одного курса, но с разными id. Таких дубликатов очень мало относительно числа курсов, но все равно рекомендуется их сортировать в следующей последовательности: по метрике (убывание) => по названию (лексикографически по возрастанию) => по возрастанию id.

Также вы можете найти так называемый submission-файл по следующему пути на мастер-ноде: `/share/submission-files/slaba02/lab02.json`. Он обладает правильной структурой и форматом, но неправильными значениями. Идея в том, что он проходит необходимые требования чекера именно по структуре, и вам не придется пытаться понять, в чем дело, а также почему чекер не принимает ваш файл.

#### Советы

Для подбора рекомендаций следует использовать меру TFIDF, а в качестве метрики для ранжирования — косинус угла между TFIDF-векторами для разных курсов.

Что такое TFIDF? TF — это term frequency: по сути, сколько раз слово встречается в этом документе. Если мы сделаем такой word count по каждому документу, то получим вектор, который как-то характеризует этот документ.

```
мама - 2
мыла - 1
раму - 1
лапу - 1
роза - 2
упала - 3
```

Если мы сравним вектора, рассчитав дистанцую между ними, то получим вывод – насколько похожи эти тексты. Назовем этот подход наивным.

Этот подход наивен, потому что мы как бы присваиваем одинаковый вес каждому слову, которое у нас есть в тексте. А что если мы попробуем как-то повысить значимость тех слов, которые часто встречаются только в этом тексте? Для этого мы посчитаем DF – document frequency: по сути, число документов, в которых есть вхождение этого слова. Мы хотим "штрафовать" слово за частое появление в документах, поэтому делаем инверсию этой величины – буква I в TFIDF. Теперь для каждого слова мы будем считать TF и делить на IDF. Так мы получим другой вектор для нашего документа. Он может быть более правильным для наших задач.

TFIDF нужно считать для описаний курсов (desc). При извлечении слов из описания словом считаем то, что состоит из латинских или кириллических букв или цифр, знаки препинания и прочие символы не учитываются.

Для поиска слов можно использовать такой код на Python (может быть проблема с распознаванием юникода).

```
regex = re.compile(u'[\w\d]{2,}', re.U)
regex.findall(string.lower())
```

Сам TFIDF реализован в Spark, писать с нуля вычисления не требуется. При вычислении TF с помощью `HashingTF` использовалось число фичей: 10000. То есть: `tf = HashingTF(10000)`.

#### Проверка

Проверка осуществляется по результатам рекомендаций текущей рекомендательной системы на eclass.cc. Для прохождения лабораторной для каждого курса, для которого строится рекомендация, должно быть пересечение рекомендованных курсов с результатами текущей системы — **не менее 20%**.

Файл необходимо положить в свою домашнюю директорию под названием: `lab02.json`. Проверка осуществляется со страницы лабы в личном кабинете. В чекере в качестве значений для курсов указаны id и доля пересечения конкретно для каждого из курсов.

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Telegram.

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, StopWordsRemover, IDF, Normalizer
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, DoubleType
import numpy as np
import re
import itertools
from pyspark.ml import Pipeline

conf = SparkConf()
conf.set("spark.app.name", "Dmitry Zh app") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
!hdfs dfs -ls /labs/slaba02/

Found 1 items
-rw-r--r--   3 hdfs hdfs   69519728 2022-01-06 18:46 /labs/slaba02/DO_record_per_line.json


In [4]:
df = spark.read.json("/labs/slaba02/DO_record_per_line.json").cache()
df.printSchema()
df.show(5, vertical=True, truncate=False)

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---+----+---+----+----+--------+
|cat|desc| id|lang|name|provider|
+---+----+---+----+----+--------+
|  0|   0|  0|   0|   0|       0|
+---+----+---+----+----+--------+



In [6]:
@f.pandas_udf(StringType())
def wordsClean(s):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    
    return s.apply(lambda i: " ".join(regex.findall(i.lower())))

In [7]:
dfClean = df.withColumn("desc", wordsClean(f.col("desc")))
dfClean.show(1, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")

In [9]:
langs=['danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian'
       , 'italian', 'norwegian', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish']
stop_words=[StopWordsRemover.loadDefaultStopWords(i) for i in langs]
stop_words=list(itertools.chain(*stop_words))

In [10]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_filtered", stopWords=stop_words)

In [11]:
tf = HashingTF(numFeatures=10000, binary=True, inputCol=swr.getOutputCol(), outputCol="words_tf")

In [12]:
idf = IDF(inputCol=tf.getOutputCol(), outputCol="words_idf")

In [13]:
normalizer = Normalizer(inputCol=idf.getOutputCol(), outputCol="words_norm")

In [14]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    tf,
    idf,
    normalizer
])

In [15]:
model = pipeline.fit(dfClean)
data = model.transform(dfClean)


In [63]:
lst=[[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
print([i[0] for i in lst])

[23126, 21617, 16627, 11556, 16704, 13702]


In [17]:
@f.udf
def sim_cos(v1,v2):
    try:
        p = 2
        return float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
    except:
        return 0

In [42]:
from pyspark.sql.window import Window
data_filter=data.select("id", f.col("name").alias("name_id"), f.col("lang").alias("lang_id"), "words_norm").where("id in (23126, 21617, 16627, 11556, 16704, 13702)")
result=data.alias("i").join(data_filter.alias("j"), (f.col("i.id") != f.col("j.id")) & (f.col("i.lang") == f.col("j.lang_id")))\
    .select(
        f.col("i.ID").alias("i"),
        f.col("j.ID").alias("j"),
        f.col("i.name").alias("name"),
        f.col("j.name_id").alias("name_id"),
        sim_cos("i.words_norm", "j.words_norm").cast(DoubleType()).alias("sim_cosine"))\
    .select("i", "j", "name", "name_id",  "sim_cosine", f.row_number().over(Window.partitionBy("j").orderBy(f.col("sim_cosine").desc(), f.col("name").asc(), f.col("i").asc())).alias("rn"))\
    .select("*").where("rn <= 15").cache()\


In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.10f}'.format
result["sim_cosine"]=pd.to_numeric(result["sim_cosine"])

In [43]:
pd_result=result.toPandas()

In [46]:
pd.set_option('display.max_colwidth', -1)
pd_result.head(100)

,i,j,name,name_id,sim_cosine,rn
0,25782,23126,Write Clean CSS using SASS - Udemy,Compass - powerful SASS library that makes your life easier,0.3448180976,1
1,23718,23126,Grunt: Automate web development tasks and save your time,Compass - powerful SASS library that makes your life easier,0.3257119167,2
2,13665,23126,The Next Step with Sass and Compass by Lisa Catalano,Compass - powerful SASS library that makes your life easier,0.1603218953,3
3,18176,23126,Automating and Speeding Up Web Development (With Gulp) by Andrew Mead,Compass - powerful SASS library that makes your life easier,0.1431876905,4
4,17499,23126,Learning Sass - Syntactically Awesome StyleSheets,Compass - powerful SASS library that makes your life easier,0.1384389234,5
5,14760,23126,Foundation 4: Incorporating Sass and Compass,Compass - powerful SASS library that makes your life easier,0.1364364301,6
6,24352,23126,"Learn How I Easily Made Over $9,975 In A Few Months On Udemy",Compass - powerful SASS library that makes your life easier,0.1345058769,7
7,5550,23126,CSS & CSS3 Beginners Course - create beautiful websites,Compass - powerful SASS library that makes your life easier,0.1321343029,8
8,23874,23126,How To Create Animated Advertising GIF banners In Photoshop,Compass - powerful SASS library that makes your life easier,0.1305340326,9
9,25709,23126,Ways To Make Money Online - I Make $9000 Monthly Doing This!,Compass - powerful SASS library that makes your life easier,0.1299858349,10


In [60]:
dct={}
for ind, row in pd_result.iterrows():
    if row["rn"]<=10:
        if row["j"] not in dct:
            dct[row["j"]]=[row["i"]]
        else:
            dct[row["j"]].append(row["i"])
dct

{23126: [25782, 23718, 13665, 18176, 17499, 14760, 24352, 5550, 23874, 25709],
 16627: [11431, 12247, 16694, 23506, 5680, 12660, 23369, 5687, 5558, 17964],
 13702: [864, 21079, 1041, 13057, 1033, 22053, 1217, 8083, 792, 1173],
 16704: [1236, 1164, 8186, 1365, 875, 8207, 1376, 1219, 1285, 8154],
 11556: [16488, 12679, 22710, 17910, 18005, 17831, 19394, 21707, 10781, 468],
 21617: [21609, 21608, 21616, 21492, 21700, 21676, 21854, 21508, 21523, 21623]}

In [61]:
import json
with open("lab02.json", "w") as outfile:
    outfile.write(json.dumps(dct))

In [62]:
json.load(open("lab02.json"))

{'23126': [25782,
  23718,
  13665,
  18176,
  17499,
  14760,
  24352,
  5550,
  23874,
  25709],
 '16627': [11431, 12247, 16694, 23506, 5680, 12660, 23369, 5687, 5558, 17964],
 '13702': [864, 21079, 1041, 13057, 1033, 22053, 1217, 8083, 792, 1173],
 '16704': [1236, 1164, 8186, 1365, 875, 8207, 1376, 1219, 1285, 8154],
 '11556': [16488, 12679, 22710, 17910, 18005, 17831, 19394, 21707, 10781, 468],
 '21617': [21609,
  21608,
  21616,
  21492,
  21700,
  21676,
  21854,
  21508,
  21523,
  21623]}